In [18]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [67]:
CATEGORICAL_FEATURES = ['Condition', 'Clouds', 'Day/Night', 'Snow_Priority']
data = pd.read_csv('../data/processed/merged_data_2.csv')

data['MINUTE'] = data['CRASH TIME'].apply(lambda x: x.split(':')[1])
data['MINUTE'] = data['MINUTE'].astype(int)
data['YEAR'] = (data['CRASH DATE'].apply(lambda x: x.split('-')[0])).astype(int)
data['MONTH'] = (data['CRASH DATE'].apply(lambda x: x.split('-')[1])).astype(int)
data['DOW'] = (data['CRASH DATE'].apply(lambda x: pd.Timestamp(x).dayofweek)).astype(int)

data['LL_X'] = np.cos(data['LATITUDE']) * np.cos(data['LONGITUDE'])
data['LL_Y'] = np.cos(data['LATITUDE']) * np.sin(data['LONGITUDE'])
data['LL_Z'] = np.sin(data['LATITUDE'])
'''ll_scaler = preprocessing.StandardScaler()
ll_scaler.fit(cleanData[["XLL","YLL", "ZLL"]])
cleanData[["XLL","YLL", "ZLL"]] = ll_scaler.transform(cleanData[["XLL","YLL", "ZLL"]])'''

data['HOUR_SIN'] = np.sin(2 * np.pi * data['HOUR'] / 23)
data['HOUR_COS'] = np.cos(2 * np.pi * data['HOUR'] / 23)
data['MONTH_SIN'] = np.sin(2 * np.pi * data['MONTH'] / 11)
data['MONTH_COS'] = np.cos(2 * np.pi * data['MONTH'] / 11)
data['DOW_SIN'] = np.sin(2 * np.pi * data['DOW'] / 6)
data['DOW_COS'] = np.cos(2 * np.pi * data['DOW'] / 6)

data = data.drop(columns=['CRASH DATE', 'CRASH TIME', 'ZIP CODE', 'geometry', 'NonPed', 'Wind',
                          'XFrom', 'YFrom', 'XTo', 'YTo', 'TRUCK_ROUTE_TYPE', 'ON STREET NAME',
                          'MINUTE', 'YEAR', 'Pressure', 'WindSpeed', 'Humidity', 'Dew', 'BOROUGH',
                          'Temperature', 'LATITUDE', 'LONGITUDE', 'HOUR', 'DOW', 'MONTH'])

data = data.dropna()

for categorical_feature in CATEGORICAL_FEATURES:
    one_hot_encoded = pd.get_dummies(data[categorical_feature], prefix=categorical_feature)
    data = data.drop(categorical_feature, axis=1)
    data = data.join(one_hot_encoded)

y = data.SEVERITY
y = y.replace(2, 1)
x = data.drop(columns=['SEVERITY'])
x, x_test, y, y_test = train_test_split(x, y, test_size=0.1, shuffle=True, random_state=42)


/home/djordje/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(auto_class_weights='Balanced', iterations=2800, learning_rate=0.4, l2_leaf_reg=3.5, depth=4, rsm=0.98, verbose=50)

In [58]:
cat.fit(x, y)

0:	learn: 0.6859326	total: 118ms	remaining: 5m 30s
50:	learn: 0.6727729	total: 6.12s	remaining: 5m 29s
100:	learn: 0.6701740	total: 12.7s	remaining: 5m 39s
150:	learn: 0.6684089	total: 18.9s	remaining: 5m 32s
200:	learn: 0.6668462	total: 25.2s	remaining: 5m 25s
250:	learn: 0.6654883	total: 31.4s	remaining: 5m 19s
300:	learn: 0.6642573	total: 37.7s	remaining: 5m 12s
350:	learn: 0.6631707	total: 44s	remaining: 5m 7s
400:	learn: 0.6620832	total: 50.2s	remaining: 5m
450:	learn: 0.6610846	total: 56.8s	remaining: 4m 55s
500:	learn: 0.6601340	total: 1m 3s	remaining: 4m 50s
550:	learn: 0.6591866	total: 1m 9s	remaining: 4m 43s
600:	learn: 0.6582871	total: 1m 15s	remaining: 4m 37s
650:	learn: 0.6574357	total: 1m 22s	remaining: 4m 31s
700:	learn: 0.6565578	total: 1m 28s	remaining: 4m 25s
750:	learn: 0.6557665	total: 1m 35s	remaining: 4m 19s
800:	learn: 0.6549446	total: 1m 41s	remaining: 4m 13s
850:	learn: 0.6541589	total: 1m 47s	remaining: 4m 7s
900:	learn: 0.6533425	total: 1m 54s	remaining: 4m
9

In [59]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

prediction = cat.predict_proba(x_test).argmax(axis=1)
accuracy = accuracy_score(y_test, prediction)
f1 = f1_score(y_test, prediction, average='micro')
f1_2 = f1_score(y_test, prediction, average='weighted')
report = classification_report(y_test, prediction)

print(f'Accuracy: \n {accuracy} \n F1: \n {f1} \n F1_2: \n {f1_2} \n Classification report: \n {report}')

Accuracy: 
 0.5854356772231168 
 F1: 
 0.5854356772231168 
 F1_2: 
 0.6299159954176627 
 Classification report: 
               precision    recall  f1-score   support

           0       0.86      0.59      0.70     58840
           1       0.25      0.58      0.35     13817

    accuracy                           0.59     72657
   macro avg       0.55      0.58      0.52     72657
weighted avg       0.74      0.59      0.63     72657



In [68]:
counter = Counter(y)
print(counter)
# define pipeline
under = RandomUnderSampler(sampling_strategy=0.35, random_state=42)
over = SMOTE(sampling_strategy=0.5, random_state=42)
steps = [('u', under), ('o', over)]
pipeline = Pipeline(steps=steps)
# transform the dataset
x_new, y_new = pipeline.fit_resample(x, y)
# summarize the new class distribution
counter = Counter(y_new)
print(counter)

Counter({0: 531324, 1: 122582})
Counter({0: 350234, 1: 175117})


In [69]:
cat = CatBoostClassifier(auto_class_weights='Balanced', iterations=2800, learning_rate=0.4, l2_leaf_reg=3.5, depth=4, rsm=0.98, verbose=50)
cat.fit(x_new, y_new)

prediction = cat.predict_proba(x_test).argmax(axis=1)
accuracy = accuracy_score(y_test, prediction)
f1 = f1_score(y_test, prediction, average='micro')
f1_2 = f1_score(y_test, prediction, average='weighted')
report = classification_report(y_test, prediction)

print(f'Accuracy: \n {accuracy} \n F1: \n {f1} \n F1_2: \n {f1_2} \n Classification report: \n {report}')

0:	learn: 0.6765941	total: 136ms	remaining: 6m 20s
50:	learn: 0.5734971	total: 5.41s	remaining: 4m 51s
100:	learn: 0.5681372	total: 10.7s	remaining: 4m 45s
150:	learn: 0.5661907	total: 16.1s	remaining: 4m 42s
200:	learn: 0.5647774	total: 21.4s	remaining: 4m 37s
250:	learn: 0.5634570	total: 26.6s	remaining: 4m 30s
300:	learn: 0.5621270	total: 31.9s	remaining: 4m 25s
350:	learn: 0.5610696	total: 37.2s	remaining: 4m 19s
400:	learn: 0.5595774	total: 42.4s	remaining: 4m 13s
450:	learn: 0.5585588	total: 47.7s	remaining: 4m 8s
500:	learn: 0.5576314	total: 53s	remaining: 4m 3s
550:	learn: 0.5567578	total: 58.2s	remaining: 3m 57s
600:	learn: 0.5561097	total: 1m 3s	remaining: 3m 52s
650:	learn: 0.5553886	total: 1m 8s	remaining: 3m 47s
700:	learn: 0.5546742	total: 1m 14s	remaining: 3m 41s
750:	learn: 0.5538408	total: 1m 19s	remaining: 3m 36s
800:	learn: 0.5529633	total: 1m 24s	remaining: 3m 30s
850:	learn: 0.5523204	total: 1m 29s	remaining: 3m 25s
900:	learn: 0.5516705	total: 1m 35s	remaining: 3m

# 2nd

In [70]:
counter = Counter(y)
print(counter)
# define pipeline
under = RandomUnderSampler(sampling_strategy=0.3, random_state=42)
over = SMOTE(sampling_strategy=0.4, random_state=42)
steps = [('u', under), ('o', over)]
pipeline = Pipeline(steps=steps)
# transform the dataset
x_new, y_new = pipeline.fit_resample(x, y)
# summarize the new class distribution
counter = Counter(y_new)
print(counter)

cat = CatBoostClassifier(auto_class_weights='Balanced', iterations=2800, learning_rate=0.4, l2_leaf_reg=3.5, depth=4, rsm=0.98, verbose=50)
cat.fit(x_new, y_new)

prediction = cat.predict_proba(x_test).argmax(axis=1)
accuracy = accuracy_score(y_test, prediction)
f1 = f1_score(y_test, prediction, average='micro')
f1_2 = f1_score(y_test, prediction, average='weighted')
report = classification_report(y_test, prediction)

print(f'Accuracy: \n {accuracy} \n F1: \n {f1} \n F1_2: \n {f1_2} \n Classification report: \n {report}')

Counter({0: 531324, 1: 122582})
Counter({0: 408606, 1: 163442})
0:	learn: 0.6784829	total: 127ms	remaining: 5m 56s
50:	learn: 0.5917999	total: 5.69s	remaining: 5m 6s
100:	learn: 0.5874652	total: 10.9s	remaining: 4m 52s
150:	learn: 0.5854113	total: 16.1s	remaining: 4m 43s
200:	learn: 0.5839719	total: 21.6s	remaining: 4m 39s
250:	learn: 0.5827001	total: 27.6s	remaining: 4m 40s
300:	learn: 0.5816442	total: 33.1s	remaining: 4m 34s
350:	learn: 0.5805284	total: 38.5s	remaining: 4m 28s
400:	learn: 0.5795064	total: 44s	remaining: 4m 23s
450:	learn: 0.5787108	total: 49.4s	remaining: 4m 17s
500:	learn: 0.5777673	total: 54.7s	remaining: 4m 11s
550:	learn: 0.5769709	total: 1m	remaining: 4m 5s
600:	learn: 0.5759858	total: 1m 5s	remaining: 3m 59s
650:	learn: 0.5751495	total: 1m 10s	remaining: 3m 53s
700:	learn: 0.5744680	total: 1m 16s	remaining: 3m 47s
750:	learn: 0.5736850	total: 1m 21s	remaining: 3m 42s
800:	learn: 0.5730204	total: 1m 26s	remaining: 3m 36s
850:	learn: 0.5723247	total: 1m 32s	remai

In [75]:
list(x.columns)

['FeelsLike',
 'Precip.',
 'StreetWidth_Min',
 'POSTED_SPEED',
 'Number_Travel_Lanes',
 'Number_Park_Lanes',
 'Number_Total_Lanes',
 'LL_X',
 'LL_Y',
 'LL_Z',
 'HOUR_SIN',
 'HOUR_COS',
 'MONTH_SIN',
 'MONTH_COS',
 'DOW_SIN',
 'DOW_COS',
 'Condition_CLD',
 'Condition_LVS',
 'Condition_RD',
 'Clouds_BKN',
 'Clouds_CLR',
 'Clouds_FEW',
 'Clouds_OVC',
 'Clouds_SCT',
 'Day/Night_D',
 'Day/Night_N',
 'Snow_Priority_C',
 'Snow_Priority_H',
 'Snow_Priority_S',
 'Snow_Priority_V']

In [76]:
from imblearn.over_sampling import SMOTENC


counter = Counter(y)
print(counter)
# define pipeline
under = RandomUnderSampler(sampling_strategy=0.25, random_state=42)
over = SMOTENC([2, 3, 4, 5, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], sampling_strategy=0.4, random_state=42)
steps = [('u', under), ('o', over)]
pipeline = Pipeline(steps=steps)
# transform the dataset
x_new, y_new = pipeline.fit_resample(x, y)
# summarize the new class distribution
counter = Counter(y_new)
print(counter)

cat = CatBoostClassifier(auto_class_weights='Balanced', iterations=2800, learning_rate=0.4, l2_leaf_reg=3.5, depth=4, rsm=0.98, verbose=50)
cat.fit(x_new, y_new)

prediction = cat.predict_proba(x_test).argmax(axis=1)
accuracy = accuracy_score(y_test, prediction)
f1 = f1_score(y_test, prediction, average='micro')
f1_2 = f1_score(y_test, prediction, average='weighted')
report = classification_report(y_test, prediction)

print(f'Accuracy: \n {accuracy} \n F1: \n {f1} \n F1_2: \n {f1_2} \n Classification report: \n {report}')

Counter({0: 531324, 1: 122582})
Counter({0: 490328, 1: 196131})
0:	learn: 0.6819559	total: 194ms	remaining: 9m 3s
50:	learn: 0.5863623	total: 8.28s	remaining: 7m 26s
100:	learn: 0.5755100	total: 15.8s	remaining: 7m
150:	learn: 0.5703712	total: 23.4s	remaining: 6m 50s
200:	learn: 0.5667128	total: 31.4s	remaining: 6m 45s
250:	learn: 0.5634066	total: 40.1s	remaining: 6m 47s
300:	learn: 0.5615419	total: 57.9s	remaining: 8m 1s
350:	learn: 0.5590629	total: 1m 19s	remaining: 9m 12s
400:	learn: 0.5576092	total: 1m 35s	remaining: 9m 31s
450:	learn: 0.5560797	total: 1m 50s	remaining: 9m 35s
500:	learn: 0.5548759	total: 2m	remaining: 9m 10s
550:	learn: 0.5536256	total: 2m 10s	remaining: 8m 52s
600:	learn: 0.5525924	total: 2m 18s	remaining: 8m 26s
650:	learn: 0.5509672	total: 2m 26s	remaining: 8m 4s
700:	learn: 0.5499273	total: 2m 39s	remaining: 7m 57s
750:	learn: 0.5489381	total: 2m 46s	remaining: 7m 35s
800:	learn: 0.5480882	total: 2m 56s	remaining: 7m 19s
850:	learn: 0.5472973	total: 3m 3s	rema